In [1]:
import os
import torch
import Self_net_architecture
import tifffile as tif
import numpy as np

In [2]:
base_path = '/home/ryuuyou/Project/self_net/data/care_liver/'

# Model

In [3]:
input_nc = 1
output_nc = 1
device=torch.device('cuda:0')
model_path = os.path.join(base_path, 'checkpoint/saved_models/deblur_net/60_3200.pkl')

In [4]:
deblur_net=Self_net_architecture.define_G(input_nc=input_nc, output_nc=output_nc, ngf=64, netG='deblur_net', device=device,use_dropout=False,norm='instance')
deblur_net.load_state_dict(torch.load(model_path))

initialize network with normal


<All keys matched successfully>

# Data

In [5]:
data_path = os.path.join(base_path, 'raw_data/xz')
data_name_list = os.listdir(data_path)
data_name_list.sort(key=lambda name:int(name.split('.')[0]))

result_save_path = os.path.join(base_path, 'reconstruction')
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)
if not os.path.exists(os.path.join(result_save_path, 'slice/')):
        os.makedirs(os.path.join(result_save_path, 'slice/'))

In [6]:
print(data_name_list)

['1.tif', '2.tif', '3.tif', '4.tif', '5.tif', '6.tif', '7.tif', '8.tif', '9.tif', '10.tif', '11.tif', '12.tif', '13.tif', '14.tif', '15.tif', '16.tif', '17.tif', '18.tif', '19.tif', '20.tif', '21.tif', '22.tif', '23.tif', '24.tif', '25.tif', '26.tif', '27.tif', '28.tif', '29.tif', '30.tif', '31.tif', '32.tif', '33.tif', '34.tif', '35.tif', '36.tif', '37.tif', '38.tif', '39.tif', '40.tif', '41.tif', '42.tif', '43.tif', '44.tif', '45.tif', '46.tif', '47.tif', '48.tif', '49.tif', '50.tif', '51.tif', '52.tif', '53.tif', '54.tif', '55.tif', '56.tif', '57.tif', '58.tif', '59.tif', '60.tif', '61.tif', '62.tif', '63.tif', '64.tif', '65.tif', '66.tif', '67.tif', '68.tif', '69.tif', '70.tif', '71.tif', '72.tif', '73.tif', '74.tif', '75.tif', '76.tif', '77.tif', '78.tif', '79.tif', '80.tif', '81.tif', '82.tif', '83.tif', '84.tif', '85.tif', '86.tif', '87.tif', '88.tif', '89.tif', '90.tif', '91.tif', '92.tif', '93.tif', '94.tif', '95.tif', '96.tif', '97.tif', '98.tif', '99.tif', '100.tif', '101.ti

In [7]:
deblur_net.eval()
res_stack=[]
for index, name in enumerate(data_name_list):
    img = tif.imread(os.path.join(data_path, name))
    img = np.expand_dims(np.expand_dims(img, axis=0), axis=0)
    img = img.astype(np.float32)
    img_tensor = torch.from_numpy(img).to(device=device).to(torch.float32)
    with torch.no_grad():
        res = deblur_net(img_tensor)
    # [x, z]
    res_npadrr = res.squeeze_(0).squeeze_(0).cpu().numpy()
    tif.imwrite(os.path.join(result_save_path, 'slice/', name), res_npadrr)
    res_stack.append(res_npadrr)
res_stack = np.asarray(res_stack, dtype=np.float32)
res_stack = np.moveaxis(res_stack, -1, 0)
tif.imwrite(os.path.join(result_save_path, 'reconstruction_stack.tif'), res_stack)